The **Laplace Transform** of a function $f(t)$ is the function of $s$ defined by the improper integral

$$ \mathcal{L}\{f(t)\} := \int_0^\infty f(t)\,e^{-st}\,\text{d}t $$

To emphasise, the left hand side is a function of the variable $s$. To make this more explicit the Laplace Transform is often written $F(s)$

$$ F(s) := \int_0^\infty f(t)\,e^{-st}\,\text{d}t $$

# Example: $f(t) = \sin(at)$

## Method 1: Using `sympy.integrals.laplace_transform`

`sympy.integrals.laplace_transform` computes the Laplace Transform of a given time-domain function $f(t)$. It takes three primary arguments:

1. **Function**: The expression $f(t)$ for which you want to find the Laplace Transform.
2. **Variable**: The time variable in the expression, typically $t$.
3. **Transform Variable**: The Laplace domain variable, often $s$.

The function returns a tuple with three elements:
1. **Transformed Function**: The Laplace Transform $F(s)$ of $f(t)$, as a symbolic expression in terms of $s$.
2. **Convergence Condition**: An expression specifying the condition on parameters for which the transform converges, often involving the imaginary part of complex constants.
3. **Validity**: A boolean value that is `True` if the transform result is valid under the given convergence conditions, and `False` if additional constraints are required.


In [2]:
from sympy.integrals import laplace_transform
from sympy.functions import sin
from sympy import symbols

t, s, a = symbols('t s a', positive=True)

f = sin(a*t)

F = laplace_transform(f, t, s)
F

(a/(a**2 + s**2), 0, True)

In [3]:
# Can extract the function F(s) alone by specifying `noconds=True`:

F = laplace_transform(f, t, s, noconds=True)
F

a/(a**2 + s**2)

In [4]:
# Or just by indexing into the tuple:

F = laplace_transform(f, t, s)[0]
F

a/(a**2 + s**2)

`F` is now just a regular SymPy expression, and we can manipulate it as such:

In [5]:
# Subbing in s=2 in F(s):

F.subs(s, 2)

a/(a**2 + 4)

In [6]:
# Differentiating F(s) with respect to s:

F.diff(s)

-2*a*s/(a**2 + s**2)**2

## Method 2: Calculating the Laplace Transform directly

**Step 1:** Calculating the indefinite integral 
$$ \int f(t)\, e^{-st} \,\text{d}t $$

In [8]:
from sympy import exp, integrate

t, s, a = symbols('t s a', real=True, positive=True)

f = sin(a*t)

indefinite_integral = integrate(f*exp(-s*t), t)
indefinite_integral

-a*cos(a*t)/(a**2*exp(s*t) + s**2*exp(s*t)) - s*sin(a*t)/(a**2*exp(s*t) + s**2*exp(s*t))

**Step 2:** Fix $K>0$ and calculate the definite integral 

$$ \int_0^K f(t)\, e^{-st} \,\text{d}t $$

In [9]:
# We could evaluate the indefinite integral between t=K and t=0 as we do on paper:

K = symbols('K', positive=True)

definite_integral = indefinite_integral.subs(t, K) - indefinite_integral.subs(t, 0)
definite_integral

-a*cos(K*a)/(a**2*exp(K*s) + s**2*exp(K*s)) + a/(a**2 + s**2) - s*sin(K*a)/(a**2*exp(K*s) + s**2*exp(K*s))

In [10]:
# Or we could calculate the definite integral directly usint `sympy.integrate` with limits:

t, s, a, K = symbols('t s a K', real=True, positive=True)

f = sin(a*t)

definite_integral = integrate(f*exp(-s*t), (t, 0, K))
definite_integral

-a*cos(K*a)/(a**2*exp(K*s) + s**2*exp(K*s)) + a/(a**2 + s**2) - s*sin(K*a)/(a**2*exp(K*s) + s**2*exp(K*s))

**Step 3:** Taking the limit $K \rightarrow \infty$ in accordance with the definition of the improper integral:

$$ \int_0^\infty f(t)\, e^{-st} \,\text{d}t := \lim_{K\rightarrow\infty} \int_0^K f(t)\, e^{-st} \,\text{d}t $$

In [11]:
from sympy import limit, oo

laplace_transform_manual = limit(definite_integral, K, oo)
laplace_transform_manual

a/(a**2 + s**2)

In [15]:
import numpy as np

def my_func(s, a, K):
    return -np.divide(a*np.cos(K*a), (a**2) * np.exp(K*s)+ s**2 * np.exp(K*s)) - np.divide(s*np.sin(K*a), (a**2) * np.exp(K*s)+ s**2 * np.exp(K*s))

In [29]:
a = 1
s = -14
for K in range(1,10):
    print(np.round(my_func(s,a,K), 4))

68617.3741
96512410565.3842
2.61833505296967e+16
-1.0555545114849004e+23
-1.7504135874305805e+29
-7.481288415543583e+34
1.5593248593332069e+41
3.1083912374517302e+47
1.7842933334061215e+53


There we have it, we've shown

$$ \mathcal{L}\{ \sin(at) \} = \frac{a}{a^2 + s^2} $$

twice, firstly by using `sympy.integrals.laplace_transform` and secondly by using `sympy.integrate` followed by `sympy.limit`

# Exercises:

Do the same as the above, but for the following functions:

1) $f(t) = \cos(at)$

2) $f(t) = \sinh(at)$

3) $f(t) = te^{at}$

**Extension:**

In a previous week I derived the standard formula for the Laplace Transform of a first derivative:

$$ \mathcal{L}\{f'(t)\} = -f(0) + sF(s) $$

Verify, using SymPy or by hand, that this formula holds for each of the functions $f(t)$ we've considered in this notebook. 

*For example* 

If $f(t) = \sin(at) $ then we can calculate $f'(t) = a\cos(at)$ and Laplace Transform this, or we could use the formula above. Do both and show that you get the same answer. 


In [43]:
# 1 - f(t) = cos(at)

from sympy.functions import cos

t, s, a = symbols('t s a', positive=True)

f = cos(a*t)

F_Q1 = laplace_transform(f, t, s)

F_Q1[0]

s/(a**2 + s**2)

In [45]:
indef_Q1 = integrate(f*exp(-s*t), t)

K = symbols('K', positive=True)

def_Q1 = integrate(f*exp(-s*t), (t, 0, K))
def_Q1


a*sin(K*a)/(a**2*exp(K*s) + s**2*exp(K*s)) - s*cos(K*a)/(a**2*exp(K*s) + s**2*exp(K*s)) + s/(a**2 + s**2)

In [48]:
laplace_manual_Q1 = limit(def_Q1, K, oo)
laplace_manual_Q1

s/(a**2 + s**2)

$$ \sinh(x) := \frac{e^x - e^{-x}}{2} $$

Normally pronounced "shine" or "sinch"

$$ \sin(x) := \frac{e^{ix} - e^{-ix}}{2i} $$

where $$ i = \sqrt{-1} $$

In [ ]:
# 2 - f(t) = sinh(at)

from sympy.functions import sinh

t, s, a = symbols('t s a', positive=True)

f = sinh(a*t)

F_Q2 = laplace_transform(f, t, s)
F_Q2[0]

a/(-a**2 + s**2)

In [57]:
laplace_transform(f, t, s)

(a/(-a**2 + s**2), a, True)

In [ ]:
#2 - manual
indef_Q2 = integrate(f * exp(-s*t), t)

K = symbols('K', positive=True)

def_Q2 = integrate(f * exp(-s*t), (t, 0, K))
def_Q2

Piecewise((-a*cosh(K*a)/(-a**2*exp(K*s) + s**2*exp(K*s)) + a/(-a**2 + s**2) - s*sinh(K*a)/(-a**2*exp(K*s) + s**2*exp(K*s)), Ne(a, s)), (K*exp(-K*s)*sinh(K*s)/2 + K*exp(-K*s)*cosh(K*s)/2 - 1/(2*s) + exp(-K*s)*cosh(K*s)/(2*s), True))

In [61]:
from sympy import Q, refine

def_Q2 = refine(def_Q2, Q.gt(s, a))
def_Q2

a*cosh(K*a)/(a**2*exp(K*s) - s**2*exp(K*s)) - a/(a**2 - s**2) + s*sinh(K*a)/(a**2*exp(K*s) - s**2*exp(K*s))

In [66]:
from sympy import Piecewise

laplace_manual_Q2 = Piecewise(
    (limit(def_Q2, K, oo), s > a)
)

laplace_manual_Q2


Piecewise((-oo/sign(-2*a + 2*s), a < s))

In [73]:
# 3 - f(t) = te^at

t, s, a = symbols('t s a', positive=True)

f = t*exp(a*t)

F_Q3 = laplace_transform(f, t, s)
F_Q3[0]


(-a + s)**(-2)

In [77]:
F_Q3

((-a + s)**(-2), a, True)

In [75]:
indef_Q3 = integrate(f*exp(-s*t), t)

K = symbols('K', positive=True)

def_Q3 = integrate(f*exp(-s*t), (t, 0, K))

In [76]:
def_Q3

Piecewise((K*a*exp(K*a)/(a**2*exp(K*s) - 2*a*s*exp(K*s) + s**2*exp(K*s)) - K*s*exp(K*a)/(a**2*exp(K*s) - 2*a*s*exp(K*s) + s**2*exp(K*s)) - exp(K*a)/(a**2*exp(K*s) - 2*a*s*exp(K*s) + s**2*exp(K*s)) + 1/(a**2 - 2*a*s + s**2), Ne(a, s)), (K**2/2, True))

# Exercises:

Do the same as the above, but for the following functions:

1) $f(t) = \cos(at)$

2) $f(t) = \sinh(at)$

3) $f(t) = te^{at}$

**Extension:**

In a previous week I derived the standard formula for the Laplace Transform of a first derivative:

$$ \mathcal{L}\{f'(t)\} = -f(0) + sF(s) $$

Verify, using SymPy or by hand, that this formula holds for each of the functions $f(t)$ we've considered in this notebook. 

*For example* 

If $f(t) = \sin(at) $ then we can calculate $f'(t) = a\cos(at)$ and Laplace Transform this, or we could use the formula above. Do both and show that you get the same answer. 


In [91]:
# Calculating the laplace transform of f' by first differentiating, them Laplace Transforming

f = sinh(a*t)

f_diff = f.diff(t)

lap_diff = laplace_transform(f_diff, t, s)

lap_diff[0]


a*s/(-a**2 + s**2)

In [94]:
# Calculating the laplace transform of f' by using the formula L(f'(t)) = -f(0) + sF(s)

ans = -f.subs(t,0) + s*laplace_transform(f, t, s)[0]
ans


a*s/(-a**2 + s**2)

In [96]:
lap_diff[0] == ans

True

In [103]:
from sympy import simplify

def does_formula_work(f):
    return (simplify(laplace_transform(f.diff(t), t, s)[0])) == (simplify(- f.subs(t, 0) + s*laplace_transform(f, t, s)[0]))

In [104]:
from sympy import cosh

f = cosh(a*t)

does_formula_work(f)

True

In [105]:
from sympy import tanh

f = tanh(a*t)

does_formula_work(f)

False